In [ ]:
import os
import sys
!{sys.executable} -m pip install opencv-python
!{sys.executable} -m pip install scikit-learn
import cv2
import numpy as np
import math
import sklearn
from sklearn import cluster
from sklearn import neighbors

In [4]:
def captura_videos_y_frames(videoName,imagesFolder):
    
    print ('\nEn la siguiente ruta se guardarán los frames del video:\n', imagesFolder,'\n')
    
    #capturamos el video
    cap = cv2.VideoCapture(videoName)
    
    #sacamos el frame rate
    frameRate = cap.get(5) 
    
    #Array que contendrá los frames del video
    frames =[]
    
    #Array que contendrá los indices del video
    nombres=[]
    
    #sacamos la longitud del video en frames
    length = int(cap.get(7))
    
    
    #avisamos si el nombre del video está mal escrito o si el archivo es erróneo
    if length==0:
        print('\n\n\n¡¡¡¡EL VIDEO NO ES CORRECTO O ESTÁ MAL ESCRITO EL NOMBRE!!!!\n\n')
        raise ValueError("El video '"+videoName+ "' es invalido o no existe")
        
        
    
    #Calculamos automáticamente el numero de centros escogiendo 1.25 clusters por minuto y le sumamos 7 más por si es muy corto
    numero_centros = int((length//frameRate)/30)+7
    
    
    #sacamos los frames del video original
    while(cap.isOpened()):
    
        frameId = cap.get(1) #numero del frame actual
    
        ret, frame = cap.read()
        if (ret != True):
            break
        
        
    
        #al final solo cojo los frames que son multiplos de 20 para no sobrecargar de información
        if (frameId % 20== 0.0):
       
            #nombre de la imagen
            filename = imagesFolder + "/image_" +  str(int(frameId)) + ".jpg"
            #escribo la imagen
            cv2.imwrite(filename, frame)
            frame= cv2.imread(filename)
            #meto el frame en la lista frames
            frames.append(frame)
            #meto el índice en la lista nombres
            nombres.append(frameId)
            
    #cierro la captura del video        
    cap.release()
    
    #avisamos si el nombre del video está mal escrito o si el archivo es erróneo
    if not frames:
        print('El video no es correcto o está mal escrito el nombre')
        
    
    else:    
        print ("Video convertido en frames!")
        print ('Número de frames --->', len(frames))
    return frames,numero_centros,nombres,frameRate


In [5]:
def calcula_histogramas(frames):
    print('\nSe han calculado los histogramas de los frames del video\n')
    colores = ['b','g','r']  
    
    #Array donde guardaremos los histogramas
    hist=[]
    
    #recorremos los frames claculando los histogramas
    for frame in frames:
        #lo hacemos para los tres colores
        feature_value=[cv2.calcHist([frame],[i],None,[16],[0,256]) for i,col in enumerate(colores)]
        #se agregan a la lista todos juntos
        hist.append(np.asarray(feature_value).flatten())
    hist=np.asarray(hist)
    
    
    # "altura y anchura" de hist
    print ("Forma del histograma (numero de frames y 256_intensidadColor*3_colores) ---> " + str(hist.shape))
    
    
    
    return hist
    
    

In [6]:
def clustering(centros,hist):
    #aplicamos kmeans a nuestro array de histogramas con nuestro número de centros calculados
    kmeans=cluster.KMeans(n_clusters=centros, n_init=10).fit(hist)
    print ("\nClustering o agrupamiento hecho!")
    return kmeans

In [7]:
def generar_resumen_frames(kmeans,hist,frames):
    print('\nGenerando resumen de frames')
    
    #Lista con los índices escogidos
    frame_indices=[]
    #Lista con los frames escogidos
    frames_resumen=[]
    
    #hist_transform es una matriz en la que cada columna i indica
    # la distancia de cada pixel al centro i
    hist_transform=kmeans.transform(hist)
    #print('histograma_transformado',hist_transform)
    
    
    
    #para cada columna o cluster
    for cluster in range(hist_transform.shape[1]):
        
        #calculamos el valor mínimo de la columna buscando el min de las filas de la traspuesta
        #y lo añadimos al array de índices
        frame_indices.append(np.argmin(hist_transform.T[cluster]))
        
    
    
    #ordenamos los frames en orden de aparición
    #en el video para no tener saltos temporales
    frame_indices=sorted(frame_indices)
    
    print ('\nSe han escogido las siguientes POSICIONES (en los frames de sampleo) para el resumen:\n', frame_indices)
    
    #metemos los frames que indican los índices en la lísta de los frames resumen
    frames_resumen=[frames[i] for i in frame_indices]
    #print(frames_resumen)
    return frames_resumen,frame_indices


In [8]:
def sacar_vecinos_fluido(videoName,frames_indices,nombres):
    
    #volvemos a capturar el video original
    cap = cv2.VideoCapture(videoName)
    frameRate = cap.get(5) #frame rate
    
    #Array que contendrá los ids de los frames del video final
    ids_finales=[]
    
    
    #sacamos los frames del video original
    while(cap.isOpened()):
    
        frameId = cap.get(1) #número del frame actual
    
        ret, frame = cap.read()
        if (ret != True):
            break
        
        
        #por cada frame del video original compruebo si su índice
        #se encuentra dentro de los índices de los frames claves antes calculados
        for pos in frame_indices:
           
            if (nombres[pos]==frameId):
                #si el frame coincide cogemos 10 vecinos adyacentes por cada lado
                if (frameId>10):
                    for i in range(-10,11):
                        filenameN = int(frameId+i)
                        
                        #Guardamos el id del frame escogido para el vídeo final
                        ids_finales.append(filenameN)
                else:
                    #hay que tener en cuenta que el frame clave no sea de los 10 primeros para no coger ningún índice negativo
                    for i in range(-frameId,frameId+11):
                        filenameN =int(frameId+i)
                        
                        #Guardamos el id del frame escogido para el vídeo final
                        ids_finales.append(filenameN)
                        
                        
    #se cierra la captura
    cap.release()
    
    return ids_finales
    

In [9]:
#Una vez obtenidos todos los ids de los frames que compondrán el video final volvemos a 
#recorrer el video para capturar los frames

def coger_frames_finales(videoName,ids_finales,imagesFolder):
    
    cap = cv2.VideoCapture(videoName)
    frameRate = cap.get(5) #frame rate
    
    #Array que contendrá los frames del video final
    framesFluid =[]
   
    while(cap.isOpened()):
    
        frameId = cap.get(1) 
    
        ret, frame = cap.read()
        if (ret != True):
            break
        
        
        #por cada frame comprobamos si está su id entre los ids seleccionados
        for t in ids_finales:
            if(frameId ==t): 
                #escribimos la imagen
                filename = imagesFolder + "/image_" +  str(t) + ".jpg"
                cv2.imwrite(filename, frame)
                frame= cv2.imread(filename)
                #la guardamos en el array
                framesFluid.append(frame)
            
                        
    #cerramos la captura
    cap.release()
    return framesFluid

In [10]:
def crear_video_resumen(framesFluid, videoName, frameRate):
    
    # Nos quedamos con el nombre del video original sin el formato
    videoSinFormato = videoName.split(".")
    
    # Obtener el tamaño del frame
    altura, anchura = framesFluid[0].shape[:2]

    # Especificar el códec
    fourcc = cv2.VideoWriter_fourcc(*'XVID')

    # Inicializar VideoWriter con los parámetros correctos
    salidaVideo = cv2.VideoWriter(videoSinFormato[0]+'-Resumen.avi', fourcc, frameRate, (anchura, altura))
    
    for frame in framesFluid:
        salidaVideo.write(frame)
    
    salidaVideo.release()
    
    print('\nVideo resumen creado!')


In [ ]:
#Ejecución de todos los métodos antes definidos


print('Bienvenido al programa de Resumen Automático de Vídeos mediante Clasificación\n\nRealizado por Alfonso Alarcón Tamayo y Juan Manuel de la Oliva Aguilar\n\nPropuesto por el profesor Miguel Ángel Martínez del Amor para el\ndpto. Ciencias de la Computación e Inteligencia Artificial de la Universidad de Sevilla 2017/18 ')

print('\n\n\nEMPEZAMOS!!!')
print('\nIntroduce el nombre del vídeo con el formato\n(se tiene que especificar la ruta si el vídeo no se encuentra en la misma carpeta que el script)\nEjemplo C:/Users/User1/Videos/video.mp4')

#capturamos el nombre del video
videoName = input()

if len(videoName)==0:
    print('\nEl nombre del vídeo no puede estar vacío, por favor ejecute el script de nuevo.')
else:
    print('\nIntroduce la ruta de la carpeta donde se guardarán los fotogramas:')
    
    #capturamos el nombre dela ruta para los frames
    imagesFolder = input()
    
    if len(imagesFolder)==0:
        print('\nLa ruta no puede estar vacía, por favor ejecute el script de nuevo.')
    else:
        print('\nConvirtiendo video en frames..\n')

        frames,numero_centros,nombres,frameRate = captura_videos_y_frames(videoName,imagesFolder)

        hist=calcula_histogramas(frames)
        print('\nEl número de centros calculados es:')
        print(numero_centros)

        kmeans=clustering(int(numero_centros),hist)

        frames_resumen,frame_indices= generar_resumen_frames(kmeans,hist,frames)

        ids_finales=sacar_vecinos_fluido(videoName,frame_indices,nombres)

        framesFluid=coger_frames_finales(videoName,ids_finales,imagesFolder)

        crear_video_resumen(framesFluid,videoName,frameRate)
    


